In [1]:
import os
os.environ['VLLM_USE_MODELSCOPE'] = "True"
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ['CUDA_VISIBLE_DEVICES'] = "1,2,0" #需要在import torch前设置
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "max_split_size_mb:64"
import torch
torch.cuda.is_available()
torch.cuda.get_device_capability()
from vllm import LLM
import torch
from modelscope import AutoTokenizer, AutoModelForCausalLM, GenerationConfig



torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
#model_name = "deepseek-ai/deepseek-llm-7b-chat"
#model_name = "/data/models/qwen-32b-chat"
model_name = "/data/models/qwen-7b-chat"
#model_name = '/data/models/qwen-7b-chat-gptq'
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(model_name, trust_remote_code=True,dtype=torch.float16,tensor_parallel_size=4) #, gpu_memory_utilization=1,quantization='gptq')
from vllm import LLM, SamplingParams
import pandas as pd
import json
from transformers import AutoModel
from numpy.linalg import norm
#model = AutoModel.from_pretrained('/data/models/jina_emb/', trust_remote_code=True) 
#model.to('cuda:2')



2024-04-14 13:16:03,067 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-04-14 13:16:03,070 - modelscope - INFO - Loading ast index from /home/userroot/.cache/modelscope/ast_indexer
2024-04-14 13:16:03,115 - modelscope - INFO - Loading done! Current index file version is 1.13.3, with md5 821edacd9c7695756b5e5583d4438f12 and a total number of 972 components indexed
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 04-14 13:16:03 config.py:767] Casting torch.bfloat16 to torch.float16.


2024-04-14 13:16:06,195	INFO worker.py:1752 -- Started a local Ray instance.


INFO 04-14 13:16:07 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='/data/models/qwen-7b-chat', tokenizer='/data/models/qwen-7b-chat', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir='/data/models/qwen-7b-chat', load_format=auto, tensor_parallel_size=4, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-14 13:16:20 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 04-14 13:16:20 selector.py:25] Using XFormers backend.
(RayWorkerVllm pid=96387) INFO 04-14 13:16:21 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
(RayWorkerVllm pid=96387) INFO 04-14 13:16:21 selector.py:25] Using XFormers backend.
(RayWorkerVllm pid=96387) INFO 04-14 13:16:22 pynccl_utils.py:45] vLLM is using nccl==2.18.1
INFO 04-14 13:16:22 pynccl_utils.py:45] vLLM is using nccl==2.18.1
INFO 04-14 13:16:26 model_runner.py:104] Loading model weights took 3.6530 GB
(RayWorkerVllm pid=96512) INFO 04-14 13:16:28 model_runner.py:104] Loading model weights took 3.6530 GB
(RayWorkerVllm pid=96618) INFO 04-14 13:16:21 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs. [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability

In [2]:

#data_path = "/home/userroot/dev/iTransformer/data/iTransformer_datasets/test_stock_news.jsonl"#
data_path = "/data/stock_datasets/stock_news.jsonl"
df_news = pd.read_json(data_path,lines=True)




In [4]:

#system_prompt = "从下面内容中总结内容，不要超过150字。"
user_prompt = "总结下面的文本，给我列出一份带有主要见解和最重要事实的要点清单。字数小于250字，不要太多。"
#system_prompt = "你是一名严谨专业的投研分析人员。着重总结出吸引眼球的要点，不要太多。开始总结吧"

system_prompt = "你是一名专业的投研分析人员，你的任务是整理出投研分析所需要的内容,先总结，然后发布评论，如果无法总结，请回答不知道。评论内容在总结内容之后，以【评论】开头。全部内容不要超过150字,不允许分段。"
#get a random number between 0 and 100

import random


for i , line in df_news.iterrows():
    if random.randint(0, 100) < 95:
        continue
    if line['content'] == '':
        continue
    news = line['content'][0:4000]
    messages = [
                

                {"role": "user", "content": f"{user_prompt}\n{news}"},
                {"role":"system", "content":f"{system_prompt}"},

            ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, return_tensors="pt",truncation='only_first')
    sampling_params = SamplingParams(temperature=0, top_p=0.95, max_tokens=500)
    key_note = llm.generate(prompt,sampling_params)[0].outputs[0].text
    line['key_note'] = key_note
    print(key_note)
    #json.dump(line.to_json(), f, ensure_ascii=False)


Processed prompts: 100%|██████████| 2/2 [00:04<00:00,  2.09s/it]


要点清单：
1. 2024年央视“3·15”晚会曝光听花酒商标侵权和不正当竞争争议。
2. 贵州茅台、泸州老窖起诉听花酒业，法院判定不侵权但构成不正当竞争，赔偿30万元和20万元。
3. 听花酒业回应承认错误，将承担责任。
4. 听花酒热衷话题营销，诉讼反而提高知名度。
5. "双激活"健康酿酒理论被质疑，生津卖点可能为差异化策略。
6. 听花酒价格高，原料和工艺声称有保健功效，但专家质疑无科学依据。
7. 张雪峰个人营销历史，包括冬虫夏草产品和极草粉片。
8. 青海春天2023年预亏，酒水业务下滑，上交所关注客户关联关系。
9. 听花酒销售遇冷，市场反应有限，经销商和电商销量有限。

【评论】听花酒的争议与其高价营销策略密切相关，商标侵权和不正当竞争的判决虽不构成侵权，但品牌形象受损。其营销手法和产品功效的宣传引发质疑，消费者和经销商对其接受度有限，反映出白酒市场对品牌积淀和故事叙述的依赖。


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.74s/it]


要点清单：
1. 贵州茅台业绩稳健，Q3营收同增14%，符合预期。
2. 销量增长及产品结构变化驱动总营收上升。
3. 系列酒占比提升，直销渠道增长亮眼。
4. 成本费用控制良好，管理费用率改善明显。
5. 预计2023-23年业绩持续增长，目标价2065.28元，评级买入。
6. 风险提示：宏观经济、消费复苏及非标产品表现。

【评论】贵州茅台三季报显示，公司业绩稳健，增长符合预期，渠道调整和产品结构优化带来积极影响。管理费用率的改善是亮点，预计未来业绩持续增长。然而，宏观经济和消费复苏等外部因素仍需关注。投资评级维持买入，目标价设置合理。


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.61s/it]


...

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.82s/it]


要点清单：
1. 贵州茅台上调53%vol飞天、五星茅台出厂价，约20%平均涨幅，预计增85-90亿收入，50-55亿利润。
2. 短期批价预计稳，受供需影响，但直销渠道增强使量价调控能力强。
3. 提价顺应消费复苏趋势，提振行业信心，预计行业高端提价将持续。
4. 2023-24年盈利预测上调，目标价2149.36元，评级为"买入"。
5. 风险提示：消费复苏、商务消费及高端批价压力。

【评论】这份分析聚焦茅台酒出厂价上调对公司业绩和行业影响，强调了短期与长期的市场反应，以及对消费复苏和估值的积极预期。同时，也提醒了可能的风险因素，显示了严谨的分析态度。


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.14s/it]


要点清单：
1. 贵州茅台Q3业绩稳健，营收、利润增长18.48%和19.09%。
2. 公司上调53%vol飞天、五星出厂价，平均上涨20%，体现市场认可。
3. 渠道改革深化，直销占比44.1%，i茅台收入增长36.8%。
4. 提价旨在控制终端价格，批价反应积极，预计后续温和上升。
5. 公司聚焦战略目标，预计未来三年EPS增长，维持"买入"评级。
6. 风险包括价格波动、新品推广、销量及政策变化。

【评论】这份报告详细分析了贵州茅台Q3业绩、提价策略和市场反应，显示公司经营稳定，改革成效显著。新管理层的决策和直销模式的推进为业绩增长注入信心。然而，市场波动和政策风险仍需关注。投资者应理性看待提价对盈利的影响，持续跟踪公司动态。


KeyboardInterrupt: 